In [22]:
from distutils.version import LooseVersion

In [23]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense

#check tensorflow version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'),'please use tensorflow version 1.1 or newer'
print('tensorflow version: {}'.format(tf.__version__))

tensorflow version: 1.3.0


In [24]:
import numpy as np
import time

with open('data/letters_source.txt','r',encoding='utf8') as f:
    source_data = f.read()

with open('data/letters_target.txt','r',encoding='utf8') as f:
    target_data = f.read()

In [25]:
# data preview
source_data.split('\n')[:10]

['bsaqq',
 'npy',
 'lbwuj',
 'bqv',
 'kial',
 'tddam',
 'edxpjpg',
 'nspv',
 'huloz',
 'kmclq']

In [26]:
target_data.split('\n')[:10]

['abqqs',
 'npy',
 'bjluw',
 'bqv',
 'aikl',
 'addmt',
 'degjppx',
 'npsv',
 'hlouz',
 'cklmq']

In [27]:
#data preprocess
def extract_character_vocab(data):
    '''
    construct mapping table
    '''
    special_words = ['<PAD>','<UNK>','<GO>','<EOS>']
    
    set_words = list(set([character for line in data.split('\n') for character in line]))
    # add the four special words into the vocabulary
    int_to_vocab = {idx:word for idx,word in enumerate(special_words+set_words)}
    vocab_to_int = {word:idx for idx,word in int_to_vocab.items()}
    
    return int_to_vocab,vocab_to_int

In [28]:
#construct the mapping table
source_int_to_letter,source_letter_to_int = extract_character_vocab(source_data)
target_int_to_letter,target_letter_to_int = extract_character_vocab(target_data)

#transform the raw data
source_int = [[source_letter_to_int.get(letter,source_letter_to_int['<UNK>'])
              for letter in line] for line in source_data.split('\n')]
target_int = [[target_letter_to_int.get(letter,target_letter_to_int['<UNK>'])
              for letter in line] + [target_letter_to_int['<EOS>']] for line in target_data.split('\n')]

In [29]:
#check the result of the transformation
source_int[:10]

[[14, 25, 18, 22, 22],
 [15, 6, 13],
 [5, 14, 9, 4, 20],
 [14, 22, 27],
 [19, 16, 18, 5],
 [23, 11, 11, 18, 24],
 [26, 11, 12, 6, 20, 6, 21],
 [15, 25, 6, 27],
 [29, 4, 5, 7, 8],
 [19, 24, 17, 5, 22]]

In [30]:
target_int[:10]

[[18, 14, 22, 22, 25, 3],
 [15, 6, 13, 3],
 [14, 20, 5, 4, 9, 3],
 [14, 22, 27, 3],
 [18, 16, 19, 5, 3],
 [18, 11, 11, 24, 23, 3],
 [11, 26, 21, 20, 6, 6, 12, 3],
 [15, 6, 25, 27, 3],
 [29, 5, 7, 4, 8, 3],
 [17, 19, 5, 24, 22, 3]]

In [31]:
#construct the model

In [32]:
#input layer
def get_inputs():
    '''
    model input tensor
    '''
    inputs = tf.placeholder(tf.int32,[None,None],name='inputs')
    targets = tf.placeholder(tf.int32,[None,None],name='targets')
    learning_rate = tf.placeholder(tf.float32,name='learning_rate')
    
    # define the maximum length of the sequence
    target_sequence_length = tf.placeholder(tf.int32,(None,),name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length,name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32,(None,),name='source_sequence_length')
    
    return inputs,targets,learning_rate,target_sequence_length,max_target_sequence_length,source_sequence_length

In [33]:
def get_encoder_layer(input_data,rnn_size,num_layers,
                     source_sequence_length,source_vocab_size,
                     encoding_embedding_size):
    '''
    construct Encoder layer
    
    params definition:
    - input_data: input tensor
    - rnn_size: rnn hidden units number
    - num_layers: the number of layers
    - source_sequence_length: the size of source data
    - encoding_embedding_size: the size of embedding
    '''
    # Encoder embedding
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data,source_vocab_size,
                                                          encoding_embedding_size)
    
    # RNN cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        return lstm_cell
    
    cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    encoder_output,encoder_state = tf.nn.dynamic_rnn(cell,encoder_embed_input,
                                                    sequence_length=source_sequence_length,dtype=tf.float32)
    return encoder_output,encoder_state

In [34]:
def process_decoder_input(data,vocab_to_int,batch_size):
    '''
    supply <GO>, remove the last character
    '''
    # cut the last character
    # tf.strided_slice(input_, start, end, strides)
    ending = tf.strided_slice(data,[0,0],[batch_size,-1],[1,1])
    decoder_input = tf.concat([tf.fill([batch_size,1],vocab_to_int['<GO>']),ending],1)
    
    return decoder_input

In [35]:
def decoding_layer(target_letter_to_int,decoding_embedding_size,num_layers,rnn_size,
                  target_sequence_length,max_target_sequence,encoder_state,decoder_input):
    '''
    construct the decoder layer
    
    @params:
    - target_letter_to_int: target data
    - decoding_embedding_size: embed size
    - num_layers: the number of RNN layers
    - rnn_size: the number of rnn units
    - target_sequence_length: target data length
    - max_target_sequence_length: the maximum length of target data sequences
    - encoder_state: the state vector of encoder RNN
    - decoder_input: decoder input
    '''
    # 1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size,decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings,decoder_input)
    
    # 2. construct the rnn cell of decoder
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        return decoder_cell
    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])
    
    # 3. construct the fully connected layer
    output_layer = Dense(target_vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))
    
    # 4. training decoder
    with tf.variable_scope('decoder'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length=target_sequence_length,time_major=False)
        # construct decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,training_helper,encoder_state,
                                                          output_layer)
        training_decoder_output, _,tmp_length = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
    
    # 5. Predicting decoder
    # share parameters with training
    with tf.variable_scope('decoder',reuse=True):
        # construct a constant tensor which has the size of batch_size
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']],dtype=tf.int32),[batch_size],name='start_tokens')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                    start_tokens,
                                                                    target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,predicting_helper,
                                                            encoder_state,
                                                            output_layer)
        predicting_decoder_output,_,tmp_length = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
        
    return training_decoder_output,predicting_decoder_output        

In [36]:
def seq2seq_model(input_data,targets,lr,target_sequence_length,
                 max_target_sequence_length,source_sequence_length,
                 source_vocab_size,target_vocab_size,encoder_embedding_size,
                 decoder_embedding_size,rnn_size,num_layers):
    
    # get the state from encoder
    _,encoder_state = get_encoder_layer(input_data,rnn_size,num_layers,
                                       source_sequence_length,source_vocab_size,
                                       encoding_embedding_size)
    # the inputs of decoder
    decoder_input = process_decoder_input(targets,target_letter_to_int,batch_size)
    
    # pass the state vector to decoder
    training_decoder_output,predicting_decoder_output = decoding_layer(target_letter_to_int,
                                                                      decoding_embedding_size,
                                                                      num_layers,
                                                                      rnn_size,
                                                                      target_sequence_length,
                                                                      max_target_sequence_length,
                                                                      encoder_state,
                                                                      decoder_input)
    return training_decoder_output,predicting_decoder_output

In [37]:
# hyperparams
# number of epochs
epochs = 60
# batch size
batch_size = 128
# rnn size
rnn_size = 50
# number of layers
num_layers = 2
# embedding_size
encoding_embedding_size = 15
decoding_embedding_size = 15
# learning rate
learning_rate = 0.001

In [38]:
#construct graph
train_graph = tf.Graph()

with train_graph.as_default():
    
    # get the inputs of the model
    input_data,targets,lr,target_sequence_length,max_target_sequence_length,source_sequence_length = get_inputs()
    
    training_decoder_output,predicting_decoder_output = seq2seq_model(input_data,
                                                                     targets,
                                                                     lr,
                                                                     target_sequence_length,
                                                                     max_target_sequence_length,
                                                                     source_sequence_length,
                                                                     len(source_letter_to_int),
                                                                     len(target_letter_to_int),
                                                                     encoding_embedding_size,
                                                                     decoding_embedding_size,
                                                                     rnn_size,
                                                                     num_layers)
    
    training_logits = tf.identity(training_decoder_output.rnn_output,'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id,name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length,max_target_sequence_length,dtype=tf.float32,
                            name='masks')
    
    with tf.name_scope('optimization'):
        
        # loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits,
                                               targets,masks)
        
        # optimizer
        optimizer = tf.train.AdamOptimizer(lr)
        
        # gradient clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad,-5.,5.),var) for grad,var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [39]:
#batches
def pad_sentence_batch(sentence_batch,pad_int):
    '''
    对batch中的序列进行补全，保证batch中的每行都有相同的sequence_length
    
    @params:
    - sentence batch
    - pad_int: <PAD>对应索引号
    '''
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int]*(max_sentence - len(sentence)) for sentence in sentence_batch]

In [42]:
def get_batches(targets,sources,batch_size,source_pad_int,target_pad_int):
    '''
    define generator to obtain batch
    '''
    for batch_i in range(0,len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i+batch_size]
        targets_batch = targets[start_i:start_i+batch_size]
        
        #pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch,source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch,target_pad_int))
        
        #calculate the length of every sample
        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))
        
        sources_lengths = []
        for source in sources_batch:
            sources_lengths.append(len(source))
        
        yield pad_targets_batch,pad_sources_batch,targets_lengths,sources_lengths

In [43]:
## Train
# split the raw dataset into train set and validation set
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# one batch for validate
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]
(valid_targets_batch,valid_sources_batch,valid_targets_lengths,valid_sources_lengths) = next(get_batches(valid_target,
                                                valid_source,batch_size,source_letter_to_int['<PAD>'],
                                                target_letter_to_int['<PAD>']))

display_step = 50 #print loss every 50 epochs

checkpoint = "trained_model.ckpt"
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_i in range(1,epochs+1):
        for batch_i,(targets_batch,sources_batch,targets_lengths,sources_lengths) in enumerate(
        get_batches(train_target,train_source,batch_size,
                   source_letter_to_int['<PAD>'],
                   target_letter_to_int['<PAD>'])):
            _,loss = sess.run(
            [train_op,cost],
            {input_data:sources_batch,
            targets:targets_batch,
            lr:learning_rate,
            target_sequence_length:targets_lengths,
            source_sequence_length:sources_lengths})
        
            if batch_i % display_step == 0:
                
                # calculate validation loss
                validation_loss = sess.run(
                [cost],
                {input_data:valid_sources_batch,
                targets:valid_targets_batch,
                lr:learning_rate,
                target_sequence_length:valid_targets_lengths,
                source_sequence_length:valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))
    saver = tf.train.Saver()
    saver.save(sess,checkpoint)
    print('model trained and saved')

Epoch   1/60 Batch    0/77 - Training Loss:  3.404  - Validation loss:  3.400
Epoch   1/60 Batch   50/77 - Training Loss:  2.936  - Validation loss:  2.927
Epoch   2/60 Batch    0/77 - Training Loss:  2.586  - Validation loss:  2.595
Epoch   2/60 Batch   50/77 - Training Loss:  2.182  - Validation loss:  2.170
Epoch   3/60 Batch    0/77 - Training Loss:  2.008  - Validation loss:  1.991
Epoch   3/60 Batch   50/77 - Training Loss:  1.776  - Validation loss:  1.750
Epoch   4/60 Batch    0/77 - Training Loss:  1.649  - Validation loss:  1.646
Epoch   4/60 Batch   50/77 - Training Loss:  1.506  - Validation loss:  1.477
Epoch   5/60 Batch    0/77 - Training Loss:  1.395  - Validation loss:  1.388
Epoch   5/60 Batch   50/77 - Training Loss:  1.304  - Validation loss:  1.252
Epoch   6/60 Batch    0/77 - Training Loss:  1.212  - Validation loss:  1.187
Epoch   6/60 Batch   50/77 - Training Loss:  1.134  - Validation loss:  1.080
Epoch   7/60 Batch    0/77 - Training Loss:  1.060  - Validation

Epoch  54/60 Batch    0/77 - Training Loss:  0.007  - Validation loss:  0.011
Epoch  54/60 Batch   50/77 - Training Loss:  0.007  - Validation loss:  0.011
Epoch  55/60 Batch    0/77 - Training Loss:  0.007  - Validation loss:  0.011
Epoch  55/60 Batch   50/77 - Training Loss:  0.007  - Validation loss:  0.011
Epoch  56/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  56/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  57/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  57/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  58/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.009
Epoch  58/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.010
Epoch  59/60 Batch    0/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  59/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  60/60 Batch    0/77 - Training Loss:  0.005  - Validation